# **Import Lib**

In [69]:
import numpy as np
import pandas as pd
from scipy import stats
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from geopy.geocoders import Nominatim
import time
from scipy import integrate

# **Data Prep**

In [70]:
trade_df=pd.read_csv('trade_df (1).csv')
countries_df=pd.read_csv('countries_1997.csv')

# **Create Adj Matrix**

In [71]:
#create adj matrix
def adj_matrix(year):
  trade_year=trade_df[trade_df['Year']==year]
  adj_matrix_im=np.zeros((210, 210))
  adj_matrix_ex=np.zeros((210, 210))
  adj_matrix_im[trade_year['Index A'], trade_year['Index B']] = trade_year['Imports of country A from country B (in current US Million $)']
  adj_matrix_ex[trade_year['Index A'], trade_year['Index B']] = trade_year['Imports of country B from country A (in current US Million $)']
  adj_matrix_vol = adj_matrix_im + adj_matrix_ex
  return adj_matrix_im, adj_matrix_ex, adj_matrix_vol

# **Map Plotting**

In [72]:
#map network function
def plot_map(adj_matrix):
  map = folium.Map(location=[0, 0], zoom_start=2)
  row_sum=np.sum(adj_matrix, axis=1)
  max_sum=np.max(row_sum)

  #creating nodes
  for i, row in countries_df.iterrows():
    html_content = f"""
        <b>{countries_df.loc[i, 'country']}</b><br>
        Trade Volume: {row_sum[i]:,.2f} Million USD
    """

    # Use IFrame to render the HTML content in the popup
    iframe = folium.IFrame(html=html_content, width=150, height=75)
    popup = folium.Popup(iframe, parse_html=True)

    folium.Circle(
        location=[row['latitude'], row['longitude']],
        #radius=np.log(row_sum[i]+1)*15000,
        #radius=sum(adj_matrix[i]),
        radius= (row_sum[i]/max_sum)*1000000,
        popup = popup,
        color='orange',
        fill=True,
        fill_color='orange',
        fill_opacity=0.8,
        #popup=folium.Popup(row['country'], parse_html=True)
    ).add_to(map)

  #creating edges
  for i in range(len(adj_matrix)):
    for j in range(i + 1, len(adj_matrix)):
      if adj_matrix[i, j] > 0:
        country_1 = countries_df.iloc[i]
        country_2 = countries_df.iloc[j]
        folium.PolyLine(
          locations=[(country_1['latitude'], country_1['longitude']),
                    (country_2['latitude'], country_2['longitude'])],
          color='blue',
          weight=0.1,
          opacity=0.7
          ).add_to(map)
  return map

# **Back Bone**

In [73]:
#diparity function
def disparity_filter_adj_matrix(adj_matrix, directed=False):
    '''
    Compute significance scores (alpha) for weighted edges in the adjacency matrix.
    Args:
        adj_matrix: Numpy array representing the adjacency matrix (either directed or undirected).
        directed: Whether the graph is directed (True) or undirected (False).

    Returns:
        A filtered adjacency matrix where only significant edges are kept.
    '''
    # Get the number of nodes (matrix is square)
    num_nodes = adj_matrix.shape[0]

    # Initialize the alpha matrix to store the significance values
    alpha_matrix = np.zeros_like(adj_matrix)

    # Undirected case
    if not directed:
        for i in range(num_nodes):
            neighbors = np.where(adj_matrix[i] != 0)[0]  # Find all neighbors (non-zero entries)
            k = len(neighbors)  # Degree of the node
            if k > 1:
                sum_weights = np.sum(np.abs(adj_matrix[i, neighbors]))  # Sum of weights of all edges
                for j in neighbors:
                    w = np.abs(adj_matrix[i, j])  # Edge weight
                    p_ij = w / sum_weights  # Probability of this edge
                    # Compute the alpha value using the disparity filter formula
                    alpha_ij = 1 - (k - 1) * integrate.quad(lambda x: (1 - x) ** (k - 2), 0, p_ij)[0]
                    alpha_matrix[i, j] = alpha_ij
                    alpha_matrix[j, i] = alpha_ij  # Symmetry for undirected graphs

    # Directed case
    else:
        for i in range(num_nodes):
            # Outgoing edges
            out_neighbors = np.where(adj_matrix[i] != 0)[0]
            k_out = len(out_neighbors)
            if k_out > 1:
                sum_weights_out = np.sum(np.abs(adj_matrix[i, out_neighbors]))
                for j in out_neighbors:
                    w = np.abs(adj_matrix[i, j])
                    p_ij_out = w / sum_weights_out
                    alpha_out = 1 - (k_out - 1) * integrate.quad(lambda x: (1 - x) ** (k_out - 2), 0, p_ij_out)[0]
                    alpha_matrix[i, j] = alpha_out

            # Incoming edges
            in_neighbors = np.where(adj_matrix[:, i] != 0)[0]
            k_in = len(in_neighbors)
            if k_in > 1:
                sum_weights_in = np.sum(np.abs(adj_matrix[in_neighbors, i]))
                for j in in_neighbors:
                    w = np.abs(adj_matrix[j, i])
                    p_ij_in = w / sum_weights_in
                    alpha_in = 1 - (k_in - 1) * integrate.quad(lambda x: (1 - x) ** (k_in - 2), 0, p_ij_in)[0]
                    alpha_matrix[j, i] = alpha_in

    return alpha_matrix

def filter_by_alpha(adj_matrix, alpha_matrix, alpha_threshold=0.15):
    '''
    Filters the original adjacency matrix based on the alpha significance threshold.
    Args:
        adj_matrix: The original weighted adjacency matrix.
        alpha_matrix: The matrix of alpha values (significance scores) from the disparity filter.
        alpha_threshold: The threshold below which edges are kept (default: 0.15).

    Returns:
        A filtered adjacency matrix with only significant edges (those with alpha < alpha_threshold).
    '''
    # Initialize the filtered adjacency matrix
    filtered_matrix = np.zeros_like(adj_matrix)

    # Retain edges where alpha is below the threshold
    filtered_matrix[alpha_matrix < alpha_threshold] = adj_matrix[alpha_matrix < alpha_threshold]

    return filtered_matrix


# **Try**

In [74]:
# sum up all  the function
# compute the year and will show the backbone of international trade network
def back_bone(year):
  adj_matrix_im, adj_matrix_ex, adj_matrix_vol = adj_matrix(year)
  alpha_matrix = disparity_filter_adj_matrix(adj_matrix_vol, directed=False)
  adj_backbone = filter_by_alpha(adj_matrix_vol, alpha_matrix)
  map_backbone=plot_map(adj_backbone)
  return map_backbone


In [75]:
# try back bone map
example_map=back_bone(2013)
example_map